In [7]:
from gensim.models import Word2Vec

from gensim.models.word2vec import LineSentence

import re

In [8]:
!pip install wikipedia-api





[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import wikipediaapi

user_agent = "MiProyectoFutbol/1.0 (contacto: elenaperez03m@gmail.com)"
wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language="es")

paginas = [
    "Fútbol", "Copa Mundial de Fútbol", "LaLiga", "Premier League",
    "Champions League", "Lionel Messi", "Cristiano Ronaldo"
]

corpus = ""

for pagina in paginas:
    page = wiki.page(pagina)
    if page.exists():
        corpus += page.text + "\n\n"  




In [10]:
sentences = re.split(r'[.!?]', corpus)  # Separar en oraciones
corpus = [re.findall(r'\b\w+\b', sentence.lower()) for sentence in sentences if sentence.strip()]


In [11]:
sg_params = {
          'vector_size' : 100,
          'window' : 5,
          'min_count' : 5,
          'sg' : 1,
          'hs' : 0,
          'negative' : 20,
          'compute_loss': True,
}

cbow_params = {
          'vector_size' : 100,
          'window' : 2,
          'min_count' : 5,
          'sg' : 0,
          'hs' : 0,
          'negative' : 20,
          'compute_loss': True,
}
#Skip Gram
w2v_sg = Word2Vec(** sg_params)
#CBOW
w2v_cbow = Word2Vec(** cbow_params)
# Entrenamiento
w2v_sg.build_vocab(corpus)
w2v_cbow.build_vocab(corpus)


w2v_sg.train(corpus, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow.train(corpus, total_examples=w2v_cbow.corpus_count, epochs=5)


w2v_sg.save('./w2v_sg_d100_mc5_w5.pkl')   # d=dimension, mc=min_count, w=window
w2v_cbow.save('./w2v_cbow_d100_mc5_w2.pkl')

In [12]:
w2v_cbow.wv.most_similar("españa")

[('13', 0.9990289211273193),
 ('celebrada', 0.9989962577819824),
 ('portugal', 0.9989461898803711),
 ('francia', 0.9989251494407654),
 ('tanda', 0.9988811016082764),
 ('ida', 0.9988773465156555),
 ('penaltis', 0.9988508224487305),
 ('eliminatoria', 0.9988504648208618),
 ('semifinales', 0.9988499283790588),
 ('suiza', 0.9988283514976501)]

In [42]:
w2v_sg.wv.most_similar("real")

[('madrid', 0.9558368921279907),
 ('atlético', 0.9437264204025269),
 ('c', 0.903626561164856),
 ('arsenal', 0.8986669778823853),
 ('athletic', 0.8883079290390015),
 ('chelsea', 0.8818196654319763),
 ('bayern', 0.8774158954620361),
 ('city', 0.8735710978507996),
 ('f', 0.867600679397583),
 ('emblema', 0.8626668453216553)]

In [14]:
sg_params = {
          'vector_size' : 300,
          'window' : 5,
          'min_count' : 2,
          'sg' : 1,
          'hs' : 0,
          'negative' : 20,
          'compute_loss': True,
}

cbow_params = {
          'vector_size' : 300,
          'window' : 10,
          'min_count' : 5,
          'sg' : 0,
          'hs' : 0,
          'negative' : 20,
          'compute_loss': True,
}
#Skip Gram
w2v_sg = Word2Vec(** sg_params)
#CBOW
w2v_cbow = Word2Vec(** cbow_params)
# Entrenamiento
w2v_sg.build_vocab(corpus)
w2v_cbow.build_vocab(corpus)


w2v_sg.train(corpus, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow.train(corpus, total_examples=w2v_cbow.corpus_count, epochs=5)


w2v_sg.save('./w2v_sg_d300_mc2_w5.pkl')   # d=dimension, mc=min_count, w=window
w2v_cbow.save('./w2v_cbow_d300_mc5_w10.pkl')

In [15]:
w2v_sg.wv.most_similar("balón")

[('oro', 0.9477055668830872),
 ('premio', 0.9355930089950562),
 ('argentino', 0.866786539554596),
 ('recibió', 0.8581168055534363),
 ('entregado', 0.8567665219306946),
 ('portugués', 0.8539156317710876),
 ('bota', 0.8538699746131897),
 ('año', 0.8518428206443787),
 ('deporte', 0.8488255739212036),
 ('puskás', 0.8477292656898499)]

In [41]:
model = Word2Vec.load("w2v_sg_d100_mc5_w5.pkl") 

def resolver_analogia(palabra1, palabra2, palabra3, modelo):
    try:
        resultado = modelo.wv.most_similar(positive=[palabra3, palabra2], negative=[palabra1], topn=1)
        palabra4, similitud = resultado[0]
        print(f"Analogía: {palabra1} - {palabra2} + {palabra3} = {palabra4} (Similitud: {similitud:.2f})")
        return palabra4
    except KeyError as e:
        print(f"¡Error! Palabra no encontrada en el vocabulario: {e}")
        return None

# Ejemplo de uso:
resolver_analogia("cristiano", "gol", "messi", model)

Analogía: cristiano - gol + messi = tiro (Similitud: 0.88)


'tiro'